In [107]:
import pandas as pd
import pandas_ml as pdml
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
#Import KNN Classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import ClusterCentroids

In [108]:
#Load data
df = pd.read_csv('data.csv');
#Replace NaN with 0
df = df.fillna(0);
df['DATE'] = pd.to_datetime(df['DATE']);
df.describe()

,Average Wind Speed,PRCP,TMAX,TMIN,Fastest 2 minute wind speed,Fastest 5 second wind speed,Fog/Ice,Heavy/Freezing Fog,Thunder,Smoke/Haze,Power Outage
count,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000,2910.000000
mean,7.274945,0.079595,59.249828,37.227835,18.562715,28.019416,0.344674,0.100344,0.053265,0.150172,0.054639
std,3.533600,0.225217,21.368136,18.345112,7.288373,17.480074,0.475343,0.300509,0.224599,0.357301,0.227314
min,0.000000,0.000000,0.000000,-30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.700000,0.000000,41.000000,25.000000,14.100000,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.930000,0.000000,62.000000,38.000000,17.000000,23.900000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.400000,0.030000,78.000000,52.000000,21.900000,31.100000,1.000000,0.000000,0.000000,0.000000,0.000000
max,22.150000,4.000000,102.000000,73.000000,83.000000,185.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [109]:
#Add columns for wind squared
df['Avg Wind Speed Squared'] = pd.Series(np.square(df['Average Wind Speed']), index=df.index)
df['5 second wind speed squared'] = pd.Series(np.square(df['Fastest 5 second wind speed']));
df['2 min wind speed squared'] = pd.Series(np.square(df['Fastest 2 minute wind speed']));
df = df[['Power Outage', 'PRCP', 'TMAX','TMIN','Fastest 5 second wind speed','5 second wind speed squared', 'Fastest 2 minute wind speed' ,'2 min wind speed squared','Average Wind Speed','Avg Wind Speed Squared','Fog/Ice', 'Heavy/Freezing Fog', 'Thunder','Smoke/Haze']];

In [110]:
X = np.array(df[['Avg Wind Speed Squared','2 min wind speed squared','5 second wind speed squared','Fog/Ice','Heavy/Freezing Fog','Thunder']]);
y = np.array(df['Power Outage']);

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 43);

In [112]:
y_test = pd.DataFrame(y_test);
X_test = pd.DataFrame(X_test);

In [113]:
#Under sample based on Cluster Centroids
sampler = ClusterCentroids({0: 600});
X_train, y_train = sampler.fit_sample(X_train, y_train);
sampler = SMOTE(random_state=12, ratio=0.5);
X_train, y_train = sampler.fit_sample(X_train, y_train);

/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:75: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [114]:
X_train = pd.DataFrame(X_train);
y_train = pd.DataFrame(y_train);
clusteredData = pd.concat([X_train, y_train], ignore_index = True, axis=1);
clusteredData.to_csv('Clustered semidata', index=False);
clusteredData.describe()

,0,1,2,3,4,5,6
count,900.000000,900.000000,900.000000,900.000000,9.000000e+02,9.000000e+02,900.000000
mean,92.383457,589.826890,1873.423354,0.399914,1.214671e-01,8.220479e-02,0.333333
std,84.681611,531.880904,3407.722414,0.424266,2.871814e-01,2.426092e-01,0.471667
min,0.000000,43.740000,65.610000,0.000000,-2.775558e-17,-1.387779e-17,0.000000
25%,30.377345,259.210000,571.210000,0.000000,0.000000e+00,0.000000e+00,0.000000
50%,64.998786,479.610000,1006.929752,0.250000,0.000000e+00,0.000000e+00,0.000000
75%,130.188100,784.000000,1672.810000,1.000000,0.000000e+00,0.000000e+00,1.000000
max,451.562500,6889.000000,34225.000000,1.000000,1.000000e+00,1.000000e+00,1.000000


In [115]:
# Random Split
#Split the data in two parts.
# powerOutages = df.loc[df['Power Outage'] == 1];
# noPowerOutages = df.loc[df['Power Outage'] == 0];

# zeroDays = noPowerOutages.sample(159);
# finalData = pd.concat([powerOutages,zeroDays]);
# finalData = finalData.sample(318);


In [116]:
#Normalize data
scaledData = preprocessing.MinMaxScaler().fit_transform(X_train.values);
X_train = pd.DataFrame(scaledData, index = X_train.index, columns = X_train.columns);
scaledData = preprocessing.MinMaxScaler().fit_transform(X_test.values);
X_test = pd.DataFrame(scaledData, index = X_test.index, columns = X_test.columns);

In [117]:
knn = KNeighborsClassifier(n_neighbors=3)

In [118]:
knn.fit(X_train, y_train);
pred = knn.predict(X_test);
accuracy = accuracy_score(y_test, pred);
print("Accuracy: ", accuracy);
from sklearn.metrics import confusion_matrix
con_mat = confusion_matrix(y_test, pred, [0, 1])
print(con_mat)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


Accuracy:  0.498439125911
[[462 447]
 [ 35  17]]
